In [1]:
#!pip install minsearch

In [2]:
import minsearch, json

In [3]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [4]:
#docs_raw

In [5]:
documents=[]
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(
    text_fields=['question','section','text'],
    keyword_fields=['course']
)

In [8]:
index.fit(documents)

In [9]:
q = 'course already started, can i still enroll'

In [10]:
boost = {'question':3.0, 'text':0.5}

results = index.search(
    query=q,
    boost_dict=boost,
    num_results = 10
)

In [11]:
results

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the cour

In [12]:
from google import genai
import os
os.environ["GOOGLE_API_KEY"] =  "YOUR KEY HERE"

In [13]:
client = genai.Client()
response = client.models.generate_content(
            model="gemini-2.0-flash", 
            contents="Explain how AI works in a few words?"
        )
print(response.text)

AI learns patterns from data to make predictions or decisions.



In [14]:
prompt_template = """
You are a course teaching assistant. 
Answer the QUESTION based on the CONTEXT from the FAQ document
Use only the facts from the CONTEXT while answering the QUESTION
If the CONTEXT does not contain the answer, output NONE
QUESTION: {question}
CONTEXT: {context}

""".strip()

In [15]:
context =""
for doc in results:
    context = context + f"section: {doc['section']}\n question: {doc['question']}\n answer: {doc['text']}\n\n " 

In [16]:
# q = "I want a beer" #check for none
q = 'course already started, can i still enroll'

In [17]:
#print(context)
prompt = prompt_template.format(question = q, context =context).strip()

In [18]:
#print(prompt)

In [19]:
#lets do it in gemini
response = client.models.generate_content(
            model="gemini-2.0-flash", 
            contents= prompt 
        )
print(response.text)

Yes, you can still join the course even if it has already started. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.
Also, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


In [61]:
def search(query):
    boost = {'question':3.0, 'text':0.5}
    
    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results = 2
        )
    return results

In [64]:
#search('course already started can i still enroll')

In [65]:
query = "how do i run kafka"
results = search(query)

In [66]:
def build_prompt(query, search_results):
    prompt_template = """
        You are a course teaching assistant. 
        Answer the QUESTION based on the CONTEXT from the FAQ document
        Use only the facts from the CONTEXT while answering the QUESTION
        If the CONTEXT does not contain the answer, output NONE
        QUESTION: {question}
        CONTEXT: {context}
        
        """.strip()
    context =""
    for doc in results:
        context = context + f"section: {doc['section']}\n question: {doc['question']}\n answer: {doc['text']}\n\n " 
    #print(context)
    prompt = prompt_template.format(question = query, context =context).strip()
    return prompt

In [67]:
search_results = search(query)
prompt = build_prompt(query, search_results)
print(prompt)

You are a course teaching assistant. 
        Answer the QUESTION based on the CONTEXT from the FAQ document
        Use only the facts from the CONTEXT while answering the QUESTION
        If the CONTEXT does not contain the answer, output NONE
        QUESTION: how do i run kafka
        CONTEXT: section: Module 6: streaming with kafka
 question: Java Kafka: How to run producer/consumer/kstreams/etc in terminal
 answer: In the project directory, run:
java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java

 section: Module 6: streaming with kafka
 question: Module “kafka” not found when trying to run producer.py
 answer: Solution from Alexey: create a virtual environment and run requirements.txt and the python files in that environment.
To create a virtual env and install packages (run only once)
python -m venv env
source env/bin/activate
pip install -r ../requirements.txt
To activate it (you'll need to run it every time you need the virtual en

In [68]:
def llm(prompt):
    #lets do it in gemini
    response = client.models.generate_content(
                model="gemini-2.0-flash", 
                contents= prompt 
            )
    print(response.text)

In [69]:
#query = "How do i run kafka"

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return(answer)

In [70]:
rag(query)

```text
In the project directory, run:
java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java
```


In [71]:
rag("How do i enroll in the course")

NONE



In [72]:
rag("course already started, can i still enroll?")

NONE



In [73]:
search_results

[{'text': 'In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java',
  'section': 'Module 6: streaming with kafka',
  'question': 'Java Kafka: How to run producer/consumer/kstreams/etc in terminal',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Solution from Alexey: create a virtual environment and run requirements.txt and the python files in that environment.\nTo create a virtual env and install packages (run only once)\npython -m venv env\nsource env/bin/activate\npip install -r ../requirements.txt\nTo activate it (you'll need to run it every time you need the virtual env):\nsource env/bin/activate\nTo deactivate it:\ndeactivate\nThis works on MacOS, Linux and Windows - but for Windows the path is slightly different (it's env/Scripts/activate)\nAlso the virtual environment should be created only to run the python file. Docker images should first all be up and running.",
  'section': 'Module 6: streaming wi